In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

admissions = pd.read_csv("admissions.csv")
model = LogisticRegression()
model.fit(admissions[["gpa"]], admissions["admit"])
labels = model.predict(admissions[['gpa']])
admissions['predicted_label']= labels
print(admissions['predicted_label'].value_counts())
print(admissions.head())
admissions["actual_label"] = admissions["admit"]

correct_predictions = admissions[admissions['actual_label'] == admissions['predicted_label']]
print(correct_predictions.head(5))
accuracy = len(correct_predictions)/ len(admissions)
print(accuracy)
pos = admissions[(admissions['predicted_label']== 1) & (admissions['actual_label'] == 1)]
true_positives = len(pos)
neg = admissions[(admissions['predicted_label']== 0) & (admissions['actual_label'] == 0)]
true_negatives = len(neg)
print(true_positives)
print(true_negatives)
true_positive_filter = (admissions["predicted_label"] == 1) & (admissions["actual_label"] == 1)
true_positives = len(admissions[true_positive_filter])
false_negatives = len(admissions[(admissions['predicted_label']==0) & (admissions['actual_label']== 1)])
sensitivity = true_positives / (true_positives + false_negatives)
print(sensitivity)
true_positive_filter = (admissions["predicted_label"] == 1) & (admissions["actual_label"] == 1)
true_positives = len(admissions[true_positive_filter])
false_negative_filter = (admissions["predicted_label"] == 0) & (admissions["actual_label"] == 1)
false_negatives = len(admissions[false_negative_filter])
true_negative_filter = (admissions["predicted_label"] == 0) & (admissions["actual_label"] == 0)
true_negatives = len(admissions[true_negative_filter])
false_positives = len(admissions[(admissions["predicted_label"] == 1)&(admissions["actual_label"] == 0)])
specificity = true_negatives / (false_positives + true_negatives)
print(specificity)





admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)

print(admissions.head())
admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)
rand_index = np.random.permutation(admissions.index)
shuffled_admissions = admissions.loc[rand_index]
train = shuffled_admissions.iloc[0:515]
test = shuffled_admissions.iloc[515: len(admissions)]
print(shuffled_admissions[0:5])
import numpy as np
np.random.seed(8)

shuffled_index = np.random.permutation(admissions.index)
shuffled_admissions = admissions.loc[shuffled_index]
train = shuffled_admissions.iloc[0:515]
test = shuffled_admissions.iloc[515:len(shuffled_admissions)]
lr = LogisticRegression()
lr.fit(train[['gpa']], train['actual_label'])
test['predicted_label'] = lr.predict(test[['gpa']])
accuracy = len(test[test['actual_label']==test['predicted_label']])/(len(test))
print(accuracy)


model = LogisticRegression()
model.fit(train[["gpa"]], train["actual_label"])
labels = model.predict(test[["gpa"]])
test["predicted_label"] = labels
matches = test["predicted_label"] == test["actual_label"]
correct_predictions = test[matches]
accuracy = len(correct_predictions) / len(test)
true_positive = len(test[(test['actual_label'] == 1) & (test['predicted_label'] ==1)])
false_negative = len(test[(test['actual_label']==1) & (test['predicted_label'] ==0)])
false_positive = len(test[(test['actual_label']==0) & (test['predicted_label'] ==1)])
true_negative = len(test[(test['actual_label']==0) & (test['predicted_label'] ==0)])
sensitivity = true_positive / (true_positive + false_negative)
specificity = true_negative / (false_positive + true_negative)
print(sensitivity, specificity)



import matplotlib.pyplot as plt
from sklearn import metrics
probabilities = lr.predict_proba(test[['gpa']])
fpr,tpr,threshholds = metrics.roc_curve(test['actual_label'], probabilities[:,1])
plt.plot(fpr,tpr)

from sklearn.metrics import roc_auc_score
probabilities = lr.predict_proba(test[['gpa']])
auc_score = roc_auc_score(test['actual_label'], probabilities[:,1])
print(auc_score)



#K-fold
admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)

shuffled_index = np.random.permutation(admissions.index)
shuffled_admissions = admissions.loc[shuffled_index]
admissions = shuffled_admissions.reset_index()
admissions.ix[0:128, 'fold'] = 1
admissions.ix[129:257, 'fold'] = 2
admissions.ix[258:386, 'fold'] = 3
admissions.ix[387:514, 'fold'] = 4
admissions.ix[515:644, 'fold'] = 5


admissions['fold'] = admissions['fold'].astype('int')

print(admissions.head())
print(admissions.tail())

lr = LogisticRegression()
train1 = admissions[admissions['fold'] != 1]
test1 = admissions[admissions['fold']== 1]
lr.fit(train1[['gpa']],train1['actual_label'])

test1['predicted_label'] = lr.predict(test1[['gpa']])

correct_predictions = test1[(test1['actual_label']) == (test1['predicted_label'])]

iteration_one_accuracy = len(correct_predictions)/len(test1)
print(iteration_one_accuracy)

#Function to trian models
fold_ids = [1,2,3,4,5]
def train_and_test(df, folds):
    fold_accuracies = []
    for fold in folds:
        lr = LogisticRegression()
        train = admissions[admissions['fold'] != fold]
        test = admissions[admissions['fold'] == fold]
        lr.fit(train[['gpa']], train['actual_label'])
        test['predicted_label']= lr.predict(test[['gpa']])
        correct_predictions = test[(test['predicted_label']) ==                         (test['actual_label'])]      
        fold_accuracies.append(len(correct_predictions)/len(test))
    return(fold_accuracies)

accuracies = train_and_test(admissions, fold_ids)
print(accuracies)
average_accuracy = np.mean(accuracies)
print(average_accuracy)

from sklearn.cross_validation import KFold
from sklearn.cross_validation import cross_val_score

admissions = pd.read_csv("admissions.csv")
admissions["actual_label"] = admissions["admit"]
admissions = admissions.drop("admit", axis=1)
kf = KFold(len(admissions), 5, shuffle=True, random_state=8)
lr = LogisticRegression()
accuracies = cross_val_score(lr, admissions[['gpa']], admissions['actual_label'], scoring='accuracy', cv=kf)
average_accuracy = sum(accuracies)/len(accuracies)
print(accuracies, average_accuracy)